In [1]:
import pandas as pd

# 读取 train_essays.csv 和 train_prompts.csv 文件
essays_df = pd.read_csv('llm-detect-ai-generated-text/train_essays.csv')
prompts_df = pd.read_csv('llm-detect-ai-generated-text/train_prompts.csv')

# 使用 prompt_id 字段将两个数据集进行连接
merged_df = essays_df.merge(prompts_df, on='prompt_id', how='left')
merged_df['text'] = merged_df['prompt_name'] + '\n\n' + merged_df['instructions'] + '\n\n' + merged_df['text']
merged_df.drop(merged_df.columns[[4, 5, 6]],axis=1, inplace=True)

merged_df.to_csv('llm-detect-ai-generated-text/train_merged.csv', index=False)

导入数据集`train_merged.csv`并对每条数据的`merge_text`项进行编码，获得每条数据的向量。

In [2]:
import pandas as pd

# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

import numpy as np
import pickle
from pathlib import Path


tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


划分训练集、测试集

In [3]:
# 读取训练集的CSV
df = pd.read_csv('./train_merged.csv')

In [4]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [5]:
df.head(3)

,id,prompt_id,text,generated
0,0059830c,0,Car-free cities\n\nWrite an explanatory essay ...,0
1,005db917,0,Car-free cities\n\nWrite an explanatory essay ...,0
2,008f63e3,0,Car-free cities\n\nWrite an explanatory essay ...,0


In [6]:
print(f"Numbers of essays: {df.id.unique().shape[0]}")

Numbers of essays: 1378


In [7]:
txtdata = []
classes = []
for row in df.itertuples():
    txtdata.append(row[3])
    classes.append(row[4])

In [8]:
from sklearn import *

In [9]:
# Split the data set using the train_test_split method
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(txtdata, classes, test_size=0.3, random_state=42)


In [10]:
print(type(train_data))
print(len(train_data))

<class 'list'>
964


Embedding

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
embeddings = model.encode(train_data, show_progress_bar=True)

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

In [12]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')
print(f'Shape of the embeddings: {embeddings.shape}')

Shape of the vectorised abstract: (768,)
Shape of the embeddings: (964, 768)


训练一个分类基线模型-LR

In [13]:
# learn logistic regression classifier using CV
#  Cs is an array of possible C values
#  cv is the number of folds
#  n_jobs=-1 means run in parallel with all cores
logreg = linear_model.LogisticRegressionCV(Cs=np.logspace(-4,4,20), cv=5, n_jobs=-1)
logreg.fit(embeddings, train_labels)

print("w=", logreg.coef_)
print("b=", logreg.intercept_)

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


w= [[-2.63919510e-05 -1.55721071e-05 -2.70387414e-05  3.48827000e-05
  -6.24415630e-06 -2.08980810e-06 -2.78018743e-05 -8.00707498e-06
   3.97779357e-05 -2.62392045e-05  6.19824998e-06 -7.31013778e-05
   6.87378048e-06 -2.95348216e-05 -3.90777416e-05  2.23001964e-05
   4.54084711e-05 -6.52699900e-06  5.18414977e-06  1.84194916e-05
  -1.52363458e-06 -1.83158987e-05  5.91796589e-05  7.53367264e-05
   3.47583616e-05 -4.46697981e-06  6.39568498e-05 -4.17950954e-06
  -1.28343937e-05 -2.75637428e-06  7.90335605e-07  7.37668521e-05
   1.05933459e-05  8.56834662e-06  9.90027540e-06 -1.11628102e-04
  -2.60703493e-05  2.19979552e-05 -1.15947976e-05 -4.12821946e-05
   4.32831488e-06 -5.17915901e-05  1.12390233e-05 -4.59398617e-05
  -1.21713954e-05 -2.66924148e-05 -4.10725676e-05  4.07016089e-05
  -2.03503291e-05 -2.04360962e-05 -3.25080498e-05 -3.67768408e-05
   2.60298378e-05 -3.59139729e-05  1.24744014e-04  1.09310718e-05
  -4.68538498e-05  3.83168421e-06  1.33337595e-05 -4.20475817e-05
  -3.30

In [14]:
embeddings = model.encode(test_data, show_progress_bar=True)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [15]:
# predict from the model
predY = logreg.predict(embeddings)

# calculate accuracy
acc = metrics.accuracy_score(test_labels, predY)
print("test accuracy=", acc)

test accuracy= 0.9975845410628019


在测试数据上进行分类

In [16]:
# 读取测试集的CSV
df_test = pd.read_csv('llm-detect-ai-generated-text/test_essays.csv')

In [17]:
df_test.head(3)

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [18]:
test_txtdata = []
test_ID = []
for row in df_test.itertuples():
    test_txtdata.append(row[3])
    test_ID.append(row[1])

In [19]:
test_embeddings = model.encode(test_txtdata, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
# 使用训练好的模型进行分类预测
print(test_embeddings)

[[-0.31552577  0.20077023  0.02336733 ... -0.85193616 -0.44328833
   0.78843325]
 [-0.16133729  0.02380942 -0.5558819  ... -1.6235547   0.17173813
   1.121677  ]
 [-1.0766127  -0.35463503 -0.3630868  ... -0.4499783   0.11158902
   1.7125299 ]]


In [21]:
# 使用训练好的模型进行分类预测
y_pred_prob = logreg.predict_proba(test_embeddings)


# 获取第一类别的概率作为预测输出
y_pred = y_pred_prob[:, 1]

In [22]:
print(y_pred_prob)

[[0.99792445 0.00207555]
 [0.99792017 0.00207983]
 [0.99792109 0.00207891]]


In [23]:
print(y_pred)

[0.00207555 0.00207983 0.00207891]


In [26]:
import csv
tmp = [['id', 'generated']]
    
    # add ID numbers for each Y
for i in range(len(test_ID)):
    tmp2 = [test_ID[i],y_pred[i]]
    tmp.append(tmp2)
        
    # write CSV file
with open("result.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerows(tmp)